In [38]:
import numpy as np
import pandas as pd

In [3]:
data = pd.read_csv("/Data2/Arun-UAV/NLP/vision_halu/haloc/train_dataset/caption/tp_test.csv")

In [4]:
data.head(2)

,Unnamed: 0,image_id,prompt,hallucinated_text,source_text,source_metadata,qa_metadata,qa_ids,annotations,id,split
0,64796,2391368,<image>Can you describe the main features of t...,"In this image, there are some boxes that conta...","In this image, there are some boxes contains f...","{'source': 'localized_narratives', 'id': 'sp_2...",[],['001006084'],"{'object': [], 'attribute': [{'attribute': {'n...",caption_4,test
1,64797,2375207,<image>Write a detailed description of the giv...,A man is wearing a white uniform. He is wearin...,A man is wearing a white uniform. He is wearin...,"{'source': 'stanford', 'id': 'sp_30231'}",[],['001021520'],"{'object': [], 'attribute': [{'attribute': {'n...",caption_11,test


In [2]:
data = pd.read_csv('/Data2/Arun-UAV/NLP/vision_halu/haloc/haloc_extension/instruct/gemini_labeled_40k.csv')

In [3]:
data.head(2)

,image_id,question,answer,candidates,hallucination_candidates,candidates_inx,hallucination_candidates_inx,image_path,question_id
0,2317351.jpg,Describe the color of the hat.,The hat is blue.,"['blue', 'hat']",['blue'],"{'blue': [(11, 15)], 'hat': [(4, 7)]}","{'blue': [(11, 15)]}",/Data2/Arun-UAV/NLP/vision_halu/visual_genome/...,e7f137be-9190-42f0-a1b0-7486e8028fad
1,2317351.jpg,Describe the color of the hat.,The hat is gray.,"['hat', 'gray']",['gray'],"{'hat': [(4, 7)], 'gray': [(11, 15)]}","{'gray': [(11, 15)]}",/Data2/Arun-UAV/NLP/vision_halu/visual_genome/...,c9903799-0507-4102-84fe-3dcd587ae9e0


In [29]:
data["candidates"] = data["candidates"].apply(lambda x: list(set(eval(x))))
data["hallucination_candidates"] = data["hallucination_candidates"].apply(lambda x: list(set(eval(x))))

In [30]:
data.head(2)

,image_path,question,answer,candidates,hallucination_candidates
0,2317351.jpg,Describe the color of the hat.,The hat is blue.,"[blue, hat]",[blue]
1,2317351.jpg,Describe the color of the hat.,The hat is gray.,"[hat, gray]",[gray]


In [37]:
import re
import unicodedata
from typing import Dict, Iterable, List, Tuple

def find_word_spans(
    text: str,
    terms: Iterable[str],
    *,
    case_sensitive: bool = False,
    whole_word: bool = False,
    overlapping: bool = False,
    normalize_unicode: bool = True,
) -> Dict[str, List[Tuple[int, int]]]:
    
    if normalize_unicode:
        text = unicodedata.normalize("NFC", text)

    # Prepare result with original (possibly unnormalized) term strings as keys
    results: Dict[str, List[Tuple[int, int]]] = {t: [] for t in terms if t}

    # Build a working list of (orig_term, norm_term)
    work_terms: List[Tuple[str, str]] = []
    for t in terms:
        if not t:
            continue
        t_norm = unicodedata.normalize("NFC", t) if normalize_unicode else t
        work_terms.append((t, t_norm))

    flags = 0 if case_sensitive else re.IGNORECASE

    for orig_term, norm_term in work_terms:
        # Escape to match literal text
        escaped = re.escape(norm_term)

        if whole_word:
            # Word boundaries only at the ends (so phrases are supported)
            # Use (?<!\w) and (?!\w) instead of \b to behave well inside lookaheads for overlapping.
            core = f"(?<!\\w){escaped}(?!\\w)"
        else:
            core = escaped

        if overlapping:
            # Lookahead to capture overlapping starts.
            # Put the core inside a capturing group so we can compute the end index.
            pattern = re.compile(f"(?=({core}))", flags)
            matches = pattern.finditer(text)
            for m in matches:
                start = m.start()
                end = start + len(m.group(1))
                results[orig_term].append((start, end))
        else:
            pattern = re.compile(core, flags)
            matches = pattern.finditer(text)
            for m in matches:
                results[orig_term].append((m.start(), m.end()))

    return results


In [4]:
data.head(2)

NameError: name 'data' is not defined

In [33]:
candid_inx = [] 
hal_inx = []

for inx, row in data.iterrows():
    spans = find_word_spans(
        text=row["answer"].lower(),
        terms=[i.lower() for i in row["candidates"]])
    candid_inx.append(spans)
    
    hal_spans = find_word_spans(
        text=row["answer"].lower(),
        terms=[i.lower() for i in row["hallucination_candidates"]])
    hal_inx.append(hal_spans)
    

In [34]:
data["candidates_inx"] = candid_inx
data["hallucination_candidates_inx"] = hal_inx

In [35]:
data.head(2)

,image_path,question,answer,candidates,hallucination_candidates,candidates_inx,hallucination_candidates_inx
0,2317351.jpg,Describe the color of the hat.,The hat is blue.,"[blue, hat]",[blue],"{'blue': [(11, 15)], 'hat': [(4, 7)]}","{'blue': [(11, 15)]}"
1,2317351.jpg,Describe the color of the hat.,The hat is gray.,"[hat, gray]",[gray],"{'hat': [(4, 7)], 'gray': [(11, 15)]}","{'gray': [(11, 15)]}"


In [36]:
data.to_csv("/Data2/Arun-UAV/NLP/vision_halu/haloc/haloc_extension/instruct/gemini_labeled_40k.csv", index=False)

In [58]:
sampled_data = pd.read_csv("/Data2/Arun-UAV/NLP/vision_halu/haloc/haloc_extension/instruct/original_sampled_20k.csv")
total_data = pd.read_csv("/Data2/Arun-UAV/NLP/vision_halu/haloc/train_dataset/instruct/tp_train.csv")

In [59]:
sampled_ids = sampled_data["id"].to_list()

In [60]:
ramian_data = total_data[~total_data["id"].isin(sampled_ids)]

In [75]:
data = pd.read_csv("/Data2/Arun-UAV/NLP/vision_halu/haloc/train_dataset/vqa/tn_test.csv")

In [76]:
ramian_data = data

In [77]:
ramian_data.head(5)

,Unnamed: 0,source_text,source_metadata,qa_metadata,qa_ids,prompt,image_id,hallucinated_text,annotations,split,id
0,96513,metal,"{'id': '001006088', 'source': 'GQA'}",{},[],"<image>The question ""Which material makes up t...",2391368,metal,"{'object': [], 'attribute': [], 'relationship'...",test,vqa_3_clean
1,96514,indoors,"{'id': '00155642', 'source': 'GQA'}",{},[],<image>Is it outdoors or indoors? A short answ...,1160027,indoors,"{'object': [], 'attribute': [], 'relationship'...",test,vqa_43_clean
2,96515,field,"{'id': '00182093', 'source': 'GQA'}",{},[],<image>Use the provided image to answer the qu...,2334081,field,"{'object': [], 'attribute': [], 'relationship'...",test,vqa_54_clean
3,96516,right,"{'id': '00194450', 'source': 'GQA'}",{},[],"<image>Given the image, answer the following q...",2405761,right,"{'object': [], 'attribute': [], 'relationship'...",test,vqa_63_clean
4,96517,left,"{'id': '00194479', 'source': 'GQA'}",{},[],<image>Is the blue helmet to the left or to th...,2405761,left,"{'object': [], 'attribute': [], 'relationship'...",test,vqa_64_clean


In [78]:
ramian_data["prompt"] = ramian_data["prompt"].apply(lambda x: x.replace("<image>",""))

In [79]:
ramian_data = ramian_data[["image_id", "prompt", "hallucinated_text", "annotations"]]

In [80]:
ramian_data.head(2)

,image_id,prompt,hallucinated_text,annotations
0,2391368,"The question ""Which material makes up the silv...",metal,"{'object': [], 'attribute': [], 'relationship'..."
1,1160027,Is it outdoors or indoors? A short answer to t...,indoors,"{'object': [], 'attribute': [], 'relationship'..."


In [81]:
all_hal_cand = []

for anot in ramian_data["annotations"]:
    res = eval(anot)
    hal_cand = []
    for val in res.values():
        for i in val:
            for j in list(i.values()):
                token =  j['name']
                hal_cand.append(token)
    all_hal_cand.append(list(set(hal_cand)))

In [82]:
ramian_data["hallucination_candidates"] = all_hal_cand
ramian_data["candidates"] = [[i] for i in ramian_data["hallucinated_text"]]


/tmp/ipykernel_1485470/1222014100.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ramian_data["hallucination_candidates"] = all_hal_cand
/tmp/ipykernel_1485470/1222014100.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ramian_data["candidates"] = [[i] for i in ramian_data["hallucinated_text"]]


In [83]:
ramian_data.drop(columns=["annotations"], inplace=True)

/tmp/ipykernel_1485470/643254796.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ramian_data.drop(columns=["annotations"], inplace=True)


In [84]:
ramian_data.head(2)

,image_id,prompt,hallucinated_text,hallucination_candidates,candidates
0,2391368,"The question ""Which material makes up the silv...",metal,[],[metal]
1,1160027,Is it outdoors or indoors? A short answer to t...,indoors,[],[indoors]


In [85]:
ramian_data.columns = ["image_path", "question", "answer", "hallucination_candidates", "candidates"]


In [86]:
ramian_data.head(2)

,image_path,question,answer,hallucination_candidates,candidates
0,2391368,"The question ""Which material makes up the silv...",metal,[],[metal]
1,1160027,Is it outdoors or indoors? A short answer to t...,indoors,[],[indoors]


In [87]:
hal_inx = []

for inx, row in ramian_data.iterrows():
    hal_spans = find_word_spans(
        text=row["answer"].lower(),
        terms=[i.lower() for i in row["candidates"]])
    hal_inx.append(hal_spans)
    

In [88]:
ramian_data["candidates_inx"] = hal_inx

/tmp/ipykernel_1485470/4104138947.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ramian_data["candidates_inx"] = hal_inx


In [89]:
ramian_data.head(2)

,image_path,question,answer,hallucination_candidates,candidates,candidates_inx
0,2391368,"The question ""Which material makes up the silv...",metal,[],[metal],"{'metal': [(0, 5)]}"
1,1160027,Is it outdoors or indoors? A short answer to t...,indoors,[],[indoors],"{'indoors': [(0, 7)]}"


In [90]:
ramian_data["image_path"] = ramian_data["image_path"].apply(lambda x: str(x)+ ".jpg")

/tmp/ipykernel_1485470/308830470.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ramian_data["image_path"] = ramian_data["image_path"].apply(lambda x: str(x)+ ".jpg")


In [91]:
ramian_data.head(2)

,image_path,question,answer,hallucination_candidates,candidates,candidates_inx
0,2391368.jpg,"The question ""Which material makes up the silv...",metal,[],[metal],"{'metal': [(0, 5)]}"
1,1160027.jpg,Is it outdoors or indoors? A short answer to t...,indoors,[],[indoors],"{'indoors': [(0, 7)]}"


In [92]:
ramian_data.shape

(5041, 6)

In [93]:
from uuid import uuid4

img_folder = "/Data2/Arun-UAV/NLP/vision_halu/visual_genome/target_images/"

ramian_data = ramian_data.rename(columns={"image_path": "image_id"})
ramian_data["image_path"] = ramian_data["image_id"].apply(lambda x: img_folder + x)
ramian_data["question_id"] = [str(uuid4()) for _ in range(len(ramian_data))]

In [94]:
ramian_data.head(2)

,image_id,question,answer,hallucination_candidates,candidates,candidates_inx,image_path,question_id
0,2391368.jpg,"The question ""Which material makes up the silv...",metal,[],[metal],"{'metal': [(0, 5)]}",/Data2/Arun-UAV/NLP/vision_halu/visual_genome/...,f80bd492-ad55-4312-90ca-3a828ad5c689
1,1160027.jpg,Is it outdoors or indoors? A short answer to t...,indoors,[],[indoors],"{'indoors': [(0, 7)]}",/Data2/Arun-UAV/NLP/vision_halu/visual_genome/...,bebd49db-22c6-4e55-bb79-c47efaa0f29d


In [ ]:
ramian_data = ramian_data.drop(["candidates", "candidates_inx"], axis=1)

In [97]:
ramian_data.head(2)

,image_id,question,answer,hallucination_candidates,candidates,image_path,question_id
0,2391368.jpg,"The question ""Which material makes up the silv...",metal,[],[metal],/Data2/Arun-UAV/NLP/vision_halu/visual_genome/...,f80bd492-ad55-4312-90ca-3a828ad5c689
1,1160027.jpg,Is it outdoors or indoors? A short answer to t...,indoors,[],[indoors],/Data2/Arun-UAV/NLP/vision_halu/visual_genome/...,bebd49db-22c6-4e55-bb79-c47efaa0f29d


In [99]:
ramian_data.to_csv("/Data2/Arun-UAV/NLP/vision_halu/benchmarks/holoc/test_vqa_tn_3200.csv", index=False)

In [100]:
ramian_data.head(2)

,image_id,question,answer,hallucination_candidates,candidates,image_path,question_id
0,2391368.jpg,"The question ""Which material makes up the silv...",metal,[],[metal],/Data2/Arun-UAV/NLP/vision_halu/visual_genome/...,f80bd492-ad55-4312-90ca-3a828ad5c689
1,1160027.jpg,Is it outdoors or indoors? A short answer to t...,indoors,[],[indoors],/Data2/Arun-UAV/NLP/vision_halu/visual_genome/...,bebd49db-22c6-4e55-bb79-c47efaa0f29d


In [22]:
import pandas as pd
data = pd.read_json("/Data2/Arun-UAV/NLP/vision_halu/testing_res/chair_llava_plus_des_embed_20_des.jsonl", lines=True)

In [23]:
data.head(2)

,question,answer,question_id,image_id,image_path,data_type,generated_captions
0,Please describe this image in detail.,"{'total_image_description': 'A close-up, high-...",0,226988,/Data2/Arun-UAV/NLP/vision_halu/benchmarks/coc...,no_datatype,The image features a woman sitting at a dining...
1,Please describe this image in detail.,{'total_image_description': 'The image capture...,1,337443,/Data2/Arun-UAV/NLP/vision_halu/benchmarks/coc...,no_datatype,The image depicts a busy city street with mult...


In [24]:
data["generated_captions"].iloc[0]

'The image features a woman sitting at a dining table, taking a picture of a delicious pizza on a plate. She is using a camera to capture the moment, likely to share the image with others or to remember the meal. \n\nThe table is surrounded by several chairs, with one chair placed near the woman and the other chairs positioned around the table. There are also a few other people in the scene, with one person standing near the top left corner of the image and another person in the background on the right side. Additionally, there is a handbag placed on the table, possibly belonging to the woman or another person in the scene.'

In [25]:
data = data[["question", "generated_captions", "image_path", "question_id", "image_id"]]

In [26]:
data.rename(columns={"generated_captions": "answer"}, inplace=True)

In [30]:
data.head(2)

,question,answer,image_path,question_id,image_id,hallucination_candidates,candidates
0,Please describe this image in detail.,The image features a woman sitting at a dining...,/Data2/Arun-UAV/NLP/vision_halu/benchmarks/coc...,0,226988,[],"[sitting, at, a, dining, table,]"
1,Please describe this image in detail.,The image depicts a busy city street with mult...,/Data2/Arun-UAV/NLP/vision_halu/benchmarks/coc...,1,337443,[],"[city, street, with, multiple, cars]"


In [28]:
data["hallucination_candidates"] = str([])

In [29]:
data["candidates"] = data["answer"].apply(lambda x: x.split(" ")[5:10])

In [ ]:
image_id	question	answer	hallucination_candidates	candidates	image_path	question_id

In [31]:
data.to_csv("/Data2/Arun-UAV/NLP/vision_halu/haloc/haloc_extension/chair_caption_gen.csv", index=False)